# Python objects

See [Optimization](Oprimization.md) for theory.

### Base theory calculations & Memory usage

In [10]:
import sys
import numpy as np
import torch

Everything in Python is an object, including primitive types like integers and strings.

In [4]:
a: int = 3
b: str = "abc"
c: list[float] = [1.2, 2.3, 3.4]
print(isinstance(a, object),isinstance(b, object),isinstance(c, object))

True True True


Compare the memory usage of a Python list, a NumPy array, and a PyTorch tensor containing the same data.

In [17]:
n = 10
py_list = [i for i in range(n)]
np_arr = np.arange(n, dtype=np.int64)
pt_tensor = torch.arange(n, dtype=torch.int64)

list_header = sys.getsizeof(py_list)
total_py_mem = list_header + sum(sys.getsizeof(obj) for obj in py_list)

print(f"--- PYTHON LIST ---")
print(f"Object header: {list_header} B")
print(f"Raw data: {total_py_mem} B")

print(f"\n--- NUMPY ARRAY ---")
print(f"Object header: {sys.getsizeof(np_arr)} B")
print(f"Raw data (.nbytes): {np_arr.nbytes} B")

print(f"\n--- PYTORCH TENSOR ---")
print(f"Object header: {sys.getsizeof(pt_tensor)} B")
print(f"Raw data: {(pt_tensor.nelement() * pt_tensor.element_size())} B")


--- PYTHON LIST ---
Object header: 184 B
Raw data: 464 B

--- NUMPY ARRAY ---
Object header: 192 B
Raw data (.nbytes): 80 B

--- PYTORCH TENSOR ---
Object header: 72 B
Raw data: 80 B


In [1]:
import numpy as np
import torch
import time
import sys

n = 10**7

py_list = list(range(n))
np_arr = np.arange(n, dtype=np.int64)
pt_tensor = torch.arange(n, dtype=torch.int64)

start = time.perf_counter()
py_sum = sum(py_list)
end = time.perf_counter()
print(f"Python List Sum: {end - start:.5f}s")


start = time.perf_counter()
np_sum = np.sum(np_arr)
end = time.perf_counter()
print(f"NumPy Array Sum: {end - start:.5f}s")


start = time.perf_counter()
pt_sum = torch.sum(pt_tensor)
end = time.perf_counter()
print(f"PyTorch Tensor Sum: {end - start:.5f}s")

Python List Sum: 0.35425s
NumPy Array Sum: 0.00719s
PyTorch Tensor Sum: 0.01617s


In [1]:
import numpy as np
import ctypes
import sys

def inspect_memory():
    # 1. Obiekty Pythonowe (rozproszone)
    py_list = [100, 101, 102]

    # 2. Tablica NumPy (ciągła)
    np_arr = np.array([100, 101, 102], dtype=np.int64)

    print(f"--- LISTA PYTHON ---")
    print(f"Adres naglowka listy: {hex(id(py_list))}")
    for i, item in enumerate(py_list):
        # id(item) to adres obiektu PyObject_int
        print(f"  Element {i} (PyObject) adres: {hex(id(item))}")

    print(f"\n--- NUMPY ARRAY ---")
    # Adres naglowka (metadane: shape, strides, etc.)
    print(f"Adres naglowka ndarray: {hex(id(np_arr))}")

    # Adres surowych danych (bufor C)
    data_address = np_arr.ctypes.data
    print(f"Adres surowych danych (Data Pointer): {hex(data_address)}")

    # Weryfikacja ciągłości (Stride)
    # Dla int64 kolejny element powinien być o 8 bajtów dalej
    for i in range(len(np_arr)):
        element_addr = data_address + (i * np_arr.strides[0])
        print(f"  Element {i} fizyczny adres w RAM: {hex(element_addr)}")

    # Podejrzenie surowej zawartości pamięci przez ctypes
    print(f"\nOdczyt surowych bajtow z RAM pod adresem {hex(data_address)}:")
    raw_bytes = (ctypes.c_char * 8).from_address(data_address)
    print(f"  Bajty pierwszego elementu: {raw_bytes.raw.hex(' ')}")

if __name__ == "__main__":
    inspect_memory()

--- LISTA PYTHON ---
Adres naglowka listy: 0x18f60a768c0
  Element 0 (PyObject) adres: 0x7ff9db03d618
  Element 1 (PyObject) adres: 0x7ff9db03d638
  Element 2 (PyObject) adres: 0x7ff9db03d658

--- NUMPY ARRAY ---
Adres naglowka ndarray: 0x18f4f2ea970
Adres surowych danych (Data Pointer): 0x18f50a7df80
  Element 0 fizyczny adres w RAM: 0x18f50a7df80
  Element 1 fizyczny adres w RAM: 0x18f50a7df88
  Element 2 fizyczny adres w RAM: 0x18f50a7df90

Odczyt surowych bajtow z RAM pod adresem 0x18f50a7df80:
  Bajty pierwszego elementu: 64 00 00 00 00 00 00 00
